In [1]:
import pandas as pd

data = pd.read_csv('data/train.csv', index_col = 0, dtype={'multiple_full_time_jobs': 'boolean', 'combined_multiple_jobs': 'boolean'}).reset_index(drop=True)
strCols = data.select_dtypes(object).columns
for i in strCols:
    data[i] = data[i].str.strip()
data = data.sample(frac=0.1)
data = data.fillna(True)
data.HIREDT = data.HIREDT.apply(pd.to_datetime, format = "%m/%d/%y")
data['daysexperience'] = (pd.to_datetime('10/7/23', format = "%m/%d/%y") - data.HIREDT ).dt.days
data_num = ['RATE', 'HRSWKD', 'daysexperience']
data_drop = ['AGY', 'HIREDT']#, 'multiple_full_time_jobs', 'combined_multiple_jobs']
data.drop(data_drop, axis=1, inplace=True)

important_features = ['daysexperience', 'RATE', 'RACE', 'SEX', 'MI', 'HRSWKD', 'JC.TITLE']


In [2]:
data_to_onehot = ['NAME', 'MI', 'JOBCLASS', 'JC.TITLE', 'RACE', 'SEX', 'EMPTYPE', 'STATENUM', 'multiple_full_time_jobs', 'combined_multiple_jobs']

In [3]:
data = data[important_features + ['ANNUAL']]
data_to_onehot = list(set(important_features) & set(data_to_onehot))

In [14]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.compose import TransformedTargetRegressor
from sklearn.decomposition import SparsePCA
from sklearn.tree import DecisionTreeRegressor
import numpy as np
import scipy as sp
#regressor = LinearRegression(n_jobs=-1) np.log10 sp.special.exp10
#regressor = TransformedTargetRegressor(
#        regressor=Ridge(alpha=1e-10), func=None, inverse_func=None
#    )
regressor = DecisionTreeRegressor(max_depth=10)

#decomposition = TruncatedSVD(n_components=10,algorithm='arpack')
decomposition = SparsePCA()
transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'infrequent_if_exist', drop='if_binary', sparse=False), data_to_onehot),
    remainder = 'passthrough',
)
pipe = Pipeline([('transformer', transformer), ('decomposition', decomposition), ('regressor', regressor)])

In [15]:
from sklearn.model_selection import train_test_split
X = data.drop('ANNUAL', axis=1)
y = data.ANNUAL

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


0.27884648211328433

In [6]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for the Decision Tree Regressor
param_grid = {
    'regressor__max_depth': range(1, 20),
    'regressor__min_samples_split': range(2, 10),
    'regressor__min_samples_leaf': range(1, 10),
}

# Instantiate the GridSearchCV object
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the GridSearchCV object
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best R-squared score: {best_score:.2f}")

/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/cleme

/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/cleme

/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/cleme

/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/cleme

/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/cleme

...

/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/cleme

/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/cleme

/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/cleme

Best parameters: {'regressor__max_depth': 5, 'regressor__min_samples_leaf': 6, 'regressor__min_samples_split': 3}
Best R-squared score: 0.13


/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/clement/.virtualenvs/jupyter/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/cleme

In [11]:
best_score

0.1324468491250334

In [7]:
X_train_reduced = pipe.fit(X_train, y_train)

In [8]:
print(decomposition.explained_variance_ratio_)

[9.99931680e-01 6.42652335e-05 3.59990407e-06 6.11111347e-08
 4.45350587e-08 3.86161669e-08 3.06698242e-08 1.40118762e-08
 1.28898849e-08 1.10117304e-08]


In [9]:
X_train_reduced

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'),
                                                  ['RACE', 'MI', 'JC.TITLE',
                                                   'SEX'])])),
                ('decomposition',
                 TruncatedSVD(algorithm='arpack', n_components=10)),
                ('regressor', DecisionTreeRegressor(max_depth=10))])

In [10]:
y_train

75269    48034.68
74734    48034.68
41181    81441.00
61570    51338.40
81528    36600.00
           ...   
78645    63500.04
70076    50724.00
1494     45631.68
444      37022.76
69158    75036.48
Name: ANNUAL, Length: 7506, dtype: float64

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(X_train_reduced,y_train)
plt.xscale('log')
#plt.yscale('log')